In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key
gmaps.configure(api_key=g_key)

In [3]:
#IMPORT CSV FROM OUTPUT AND READ
cities_output = "../output_data/cities.csv"

cities_df = pd.read_csv(cities_output)

cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mogadishu,2.0371,45.3438,78.80,65,90,9.22,SO,1620282929
1,Borogontsy,62.6683,131.1683,45.07,58,100,6.42,RU,1620282929
2,Avarua,-21.2078,-159.7750,77.00,78,90,5.75,CK,1620282929
3,Cape Town,-33.9258,18.4232,59.00,88,40,23.02,ZA,1620282929
4,San Quintín,30.4833,-115.9500,60.69,87,0,3.58,MX,1620282930
...,...,...,...,...,...,...,...,...,...
591,Xai-Xai,-25.0519,33.6442,72.57,67,87,2.01,MZ,1620283059
592,Hannibal,39.7084,-91.3585,54.00,50,75,3.44,US,1620282960
593,Scarborough,54.2797,-0.4044,39.00,79,11,1.01,GB,1620283059
594,Muisne,0.6000,-80.0333,76.39,88,100,6.17,EC,1620283059


In [11]:
humidity = cities_df["Humidity"].astype(float)
locations = cities_df[["Lat", "Lng"]]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, 
                                 max_intensity=100,point_radius=2)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [35]:
#CREATE IDEAL WEATHER DATAFRAME
hotel_df = cities_df.loc[(cities_df["Max Temp"] > 75)  
                        & (cities_df["Cloudiness"] < 10) 
                        & (cities_df["Wind Speed"] < 5)]
hotel_df = hotel_df.dropna(how="any")

hotel_df.reset_index(inplace=True)

del hotel_df["index"]

hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Boa Vista,2.8197,-60.6733,75.20,88,0,3.44,BR,1620282952
1,Esna,25.2934,32.5540,89.60,19,0,4.61,EG,1620282966
2,Abnūb,27.2696,31.1511,98.13,10,0,3.83,EG,1620282990
3,Tabas,33.5959,56.9244,82.85,24,0,4.52,IR,1620282999
4,Caibarién,22.5206,-79.4669,75.20,88,1,3.44,CU,1620283001
5,Road Town,18.4167,-64.6167,78.01,87,3,1.01,VG,1620283012
6,Morondava,-20.2833,44.2833,76.12,46,4,3.38,MG,1620283016
7,Sambava,-14.2667,50.1667,82.87,71,0,3.67,MG,1620283022
8,Sagua la Grande,22.8067,-80.0756,75.20,88,1,3.44,CU,1620283034
9,Ambilobe,-13.2000,49.0500,86.00,63,0,3.02,MG,1620283049


In [44]:
hotels = []

for i in range(len(ideal_df)):
    lat = hotel_df.loc[i]["Lat"]
    lng = hotel_df.loc[i]["Lng"]
    
    params = {"location": f"{lat},{lng}",
             "radius": 5000,
             "types": "hotel",
             "key": g_key}
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(url, params=params)
    jsn = requested.json()
    try:
        hotels.append(jsn['results'][0]['name'])
    except:
        hotels.append("")
        
hotel_df["Hotel Name"] = hotels
hotel_df = hotel_df.dropna(how="any")
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Boa Vista,2.8197,-60.6733,75.20,88,0,3.44,BR,1620282952,Boa Vista
1,Esna,25.2934,32.5540,89.60,19,0,4.61,EG,1620282966,Esna
2,Abnūb,27.2696,31.1511,98.13,10,0,3.83,EG,1620282990,El-Hamam
3,Tabas,33.5959,56.9244,82.85,24,0,4.52,IR,1620282999,Tabas
4,Caibarién,22.5206,-79.4669,75.20,88,1,3.44,CU,1620283001,Caibarién
5,Road Town,18.4167,-64.6167,78.01,87,3,1.01,VG,1620283012,Road Town
6,Morondava,-20.2833,44.2833,76.12,46,4,3.38,MG,1620283016,Morondava
7,Sambava,-14.2667,50.1667,82.87,71,0,3.67,MG,1620283022,Sambava
8,Sagua la Grande,22.8067,-80.0756,75.20,88,1,3.44,CU,1620283034,Sagua La Grande
9,Ambilobe,-13.2000,49.0500,86.00,63,0,3.02,MG,1620283049,Ambilobe


In [46]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [47]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))